
이미지 총 1221  
train : 976  
valid : 122  
test : 123  
  
시작 전 한번 실행  
> poetry install  
  
_path : data 경로



In [91]:
import numpy as np
import os
from sklearn.model_selection import StratifiedGroupKFold
import pandas as pd
from tqdm import tqdm
import shutil
from glob import glob

In [92]:
_path = './data'
test_name = 'valid'
if 'test' in os.listdir(_path):
    test_name = 'test'

print(test_name)


test


In [93]:
train_abnormal_file = os.path.join(_path, 'train-abnormal.csv')
train_acl_file = os.path.join(_path, "train-acl.csv")
train_meniscus_file = os.path.join(_path, "train-meniscus.csv")

test_abnormal_file = os.path.join(_path, f"{test_name}-abnormal.csv")
test_acl_file = os.path.join(_path, f"{test_name}-acl.csv")
test_meniscus_file = os.path.join(_path, f"{test_name}-meniscus.csv")
test_abnormal_file

train_abnormal_df = pd.read_csv(train_abnormal_file, names=['name', 'abnormal'])
train_acl_df = pd.read_csv(train_acl_file, names=['name', 'acl'])
train_meniscus_df = pd.read_csv(train_meniscus_file, names=['name', 'meniscus'])

test_abnormal_df = pd.read_csv(test_abnormal_file, names=['name', 'abnormal'])
test_acl_df = pd.read_csv(test_acl_file, names=['name', 'acl'])
test_meniscus_df = pd.read_csv(test_meniscus_file, names=['name', 'meniscus'])

abnormal_df  = pd.concat([train_abnormal_df, test_abnormal_df], axis=0)
acl_df  = pd.concat([train_acl_df, test_acl_df], axis=0)
meniscus_df  = pd.concat([train_meniscus_df, test_meniscus_df], axis=0)

df = pd.concat([abnormal_df, acl_df['acl'],meniscus_df['meniscus']], axis=1).sort_values('name')
df


,name,abnormal,acl,meniscus
0,0,1,0,0
1,1,1,1,1
2,2,1,0,0
3,4,1,0,0
4,5,1,0,1
...,...,...,...,...
1094,1245,1,1,1
1095,1246,1,1,1
1096,1247,1,0,1
122,1248,1,1,1


In [94]:
with open('delete_list.txt', 'r') as f:
    delete_list =[int(line.rstrip('\n')[:4]) for line in f.readlines()]
# delete_list
idx = df[df['name'].isin(delete_list)].index

df = df[~df['name'].isin(delete_list)]
df

,name,abnormal,acl,meniscus
0,0,1,0,0
1,1,1,1,1
2,2,1,0,0
3,4,1,0,0
4,5,1,0,1
...,...,...,...,...
1094,1245,1,1,1
1095,1246,1,1,1
1096,1247,1,0,1
122,1248,1,1,1


In [95]:
classes = {0 : [0,0,0], 1 : [0,0,1], 2 : [0,1,0], 3 : [0,1,1], 4 : [1,0,0], 5 : [1,0,1], 6 : [1,1,0], 7 : [1,1,1]}
# [abnormal, acl, meniscus]

df['class'] = [[k for k, v in classes.items() 
                        if v == [df['abnormal'].iloc[i], df['acl'].iloc[i], df['meniscus'].iloc[i]]][0]
                        for i in range(df.shape[0])]
df

,name,abnormal,acl,meniscus,class
0,0,1,0,0,4
1,1,1,1,1,7
2,2,1,0,0,4
3,4,1,0,0,4
4,5,1,0,1,5
...,...,...,...,...,...
1094,1245,1,1,1,7
1095,1246,1,1,1,7
1096,1247,1,0,1,5
122,1248,1,1,1,7


In [96]:
all = df.to_numpy()
X = np.ones(all.shape[0])
y = np.array([v[-1] for v in all])
groups = np.array([v[0] for v in all])
# groups
seed = 1024
fold_num = 5 # 0 ~ 9

cv = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=seed)

train_fold = []
test_fold = []

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])
    train_fold.append(groups[train_idx])
    test_fold.append(groups[val_idx])
print(train_fold[fold_num].shape)
print(test_fold[fold_num].shape)

TRAIN: [   0    1    4 ... 1247 1248 1249]
  [4 7 4 ... 5 7 5]
 TEST: [   2   13   21   30   39   56   63   75   85   91  100  118  125  129
  144  149  166  169  179  192  205  219  228  238  253  260  271  276
  287  299  305  321  328  342  352  364  367  378  387  405  408  413
  428  432  451  452  462  474  486  496  508  522  532  542  550  560
  569  586  596  598  608  628  643  652  658  666  668  681  692  710
  713  724  734  752  753  765  779  790  800  803  815  830  843  854
  862  869  878  891  899  911  923  940  950  952  961  975  976  984
  999 1008 1026 1041 1042 1054 1055 1074 1086 1093 1104 1114 1126 1135
 1150 1152 1167 1179 1192 1196 1205 1218 1228 1244]
  [4 4 0 5 5 4 5 4 4 5 4 0 5 4 0 0 6 4 7 4 6 4 0 0 7 5 4 4 4 0 7 7 7 0 4 4 4
 5 0 4 5 4 0 5 4 5 0 5 6 4 5 7 0 0 4 4 5 5 5 6 0 5 4 4 0 5 4 4 7 6 4 7 4 7
 5 4 5 4 4 4 4 5 5 5 5 6 0 5 4 6 5 4 5 4 5 4 4 0 7 0 5 7 4 6 4 6 0 0 4 4 5
 0 0 0 4 6 7 7 7 6 7 5]
TRAIN: [   0    1    2 ... 1247 1248 1249]
  [4 7 4 ... 5 7

In [97]:
# check distribution
from collections import Counter
import pandas as pd

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0 
    
    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'test - fold{fold_ind}')

categories = [i for i in range(8)]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

,0,1,2,3,4,5,6,7
training set,19.66%,0.00%,0.00%,0.00%,36.36%,23.18%,8.52%,12.29%
train - fold0,19.75%,0.00%,0.00%,0.00%,36.49%,23.02%,8.46%,12.28%
test - fold0,18.85%,0.00%,0.00%,0.00%,35.25%,24.59%,9.02%,12.30%
train - fold1,19.02%,0.00%,0.00%,0.00%,36.31%,23.84%,8.83%,12.01%
test - fold1,25.41%,0.00%,0.00%,0.00%,36.89%,17.21%,5.74%,14.75%
train - fold2,19.49%,0.00%,0.00%,0.00%,37.16%,22.86%,8.38%,12.11%
test - fold2,21.14%,0.00%,0.00%,0.00%,29.27%,26.02%,9.76%,13.82%
train - fold3,20.13%,0.00%,0.00%,0.00%,35.88%,23.04%,8.29%,12.66%
test - fold3,15.45%,0.00%,0.00%,0.00%,40.65%,24.39%,10.57%,8.94%
train - fold4,20.29%,0.00%,0.00%,0.00%,36.31%,22.93%,9.01%,11.46%


In [98]:

train_id = train_fold[fold_num]
test_id = test_fold[fold_num]

In [99]:
before_test_path = os.path.join(_path, test_name)
train_path = os.path.join(_path, 'train')

for plane in os.listdir(before_test_path):
    if plane[0] == '.':
        continue
    print(plane)

    now_path = os.path.join(before_test_path, plane)
    des_path = os.path.join(train_path, plane)
    
    if not os.path.exists(des_path):
        os.makedirs(des_path)
    
    for num in tqdm(os.listdir(now_path)):
        shutil.move(
            os.path.join(now_path, num),
            os.path.join(des_path, num)
        )
if os.path.exists(os.path.join(_path, 'valid')):
    shutil.rmtree(os.path.join(_path, 'valid'))

axial


100%|██████████| 123/123 [00:00<00:00, 1069.56it/s]


coronal


100%|██████████| 123/123 [00:00<00:00, 809.49it/s]


sagittal


100%|██████████| 123/123 [00:00<00:00, 865.03it/s]


In [100]:
with open('delete_list.txt', 'r') as f:
    delete_npy_list =[line.rstrip('\n') for line in f.readlines()]

for plane in os.listdir(train_path):
    if plane[0] == '.':
        continue
    print(plane)

    now_path = os.path.join(train_path, plane)

    for num in tqdm(os.listdir(now_path)):
        if num[0] == '.':
            continue
        if num in delete_npy_list:
            os.remove(os.path.join(now_path, num))

axial


100%|██████████| 1222/1222 [00:00<00:00, 610753.04it/s]


coronal


100%|██████████| 1222/1222 [00:00<00:00, 1222088.58it/s]


sagittal


100%|██████████| 1222/1222 [00:00<00:00, 611992.77it/s]


In [101]:
test_path = os.path.join(_path, 'test')
if not os.path.exists(test_path):
    os.makedirs(test_path)

for plane in os.listdir(train_path):

    if plane[0] == '.':
        continue
    print(plane)

    now_path = os.path.join(train_path, plane)
    des_test_path = os.path.join(test_path, plane)
    if not os.path.exists(des_test_path):
        os.makedirs(des_test_path)

    for num in tqdm(test_id):
        file_name = str(num).zfill(4) + '.npy'
        shutil.move(
            os.path.join(now_path, file_name),
            os.path.join(des_test_path, file_name)
        )

axial


100%|██████████| 123/123 [00:00<00:00, 523.40it/s]


coronal


100%|██████████| 123/123 [00:00<00:00, 808.77it/s]


sagittal


100%|██████████| 123/123 [00:00<00:00, 1000.06it/s]


In [102]:
csv_list = glob(os.path.join(_path, '*.csv'))
for csv_file in csv_list:
    os.remove(csv_file)

In [103]:
def name_change(x):
    return str(x).zfill(4)

train_line = [line for line in all if line[0] in train_id]
test_line = [line for line in all if line[0] in test_id]
print(len(train_line))
print(len(test_line))
train_csv = pd.DataFrame(train_line, columns=df.columns)
train_csv['name'] = train_csv['name'].apply(name_change)
valid_csv = pd.DataFrame(test_line, columns=df.columns)
valid_csv['name'] = valid_csv['name'].apply(name_change)

t_abnormal = train_csv[['name', 'abnormal']]
t_abnormal.to_csv(os.path.join(_path, "train-abnormal.csv"), header=False, index=False)
t_acl = train_csv[['name', 'acl']]
t_acl.to_csv(os.path.join(_path, "train-acl.csv"), header=False, index=False)
t_meniscus = train_csv[['name', 'meniscus']]
t_meniscus.to_csv(os.path.join(_path, "train-meniscus.csv"), header=False, index=False)

test_abnormal = valid_csv[['name', 'abnormal']]
test_abnormal.to_csv(os.path.join(_path, "test-abnormal.csv"), header=False, index=False)
test_acl = valid_csv[['name', 'acl']]
test_acl.to_csv(os.path.join(_path, "test-acl.csv"), header=False, index=False)
test_meniscus = valid_csv[['name', 'meniscus']]
test_meniscus.to_csv(os.path.join(_path, "test-meniscus.csv"), header=False, index=False)

1098
123
